In [1]:
import os
import numpy as np
import pandas as pd
import nitools as nt
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from SUITPy import flatmap
import PcmPy as pcm

import selective_recruitment.plotting as plotting
import selective_recruitment.recruite_ana as ra
import selective_recruitment.globals as gl
import selective_recruitment.scripts.script_prep_sc as ss
import selective_recruitment.scripts.script_wm_sanity_checks as sa

import Functional_Fusion.dataset as fdata
import Functional_Fusion.atlas_map as am
from statsmodels.stats.anova import AnovaRM  # perform F test
from selective_recruitment.scripts.script_roi_differences import norm_within_category,prep_roi_comparison, plot_roi_differences

# import warnings
# warnings.filterwarnings('ignore')


wkdir = 'A:\data\Cerebellum\CerebellumWorkingMemory\selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/srv/diedrichsen/data/Cerebellum/CerebellumWorkingMemory/selective_recruit'
if not Path(wkdir).exists():
    wkdir = '/Users/jdiedrichsen/Data/wm_cerebellum/selective_recruit'

# Difference between ROIs in the Demand network

## Without anterior-posterior split

In [2]:
df_path = os.path.join(wkdir, "ROI_NettekovenSym68c32_conn_reg.tsv")
D = pd.read_csv(df_path, sep="\t")
D, cond_map = prep_roi_comparison(D)

In [3]:
D = norm_within_category(D, category=['roi_name','sn'], value='Y', norm='mean')
plt.figure(figsize=(12,8))
# Define styles and colors
d1 = (1,0)
d2 = (3,3)
red =(0.8,0.2,0.2)
gray = (0.5,0.5,0.5)
lb = (0.2,0.5,1.0)
db = (0.0,0.1,0.6)
# pal = [red,gray,lb,db,red,gray,lb,db]

# ax = sns.lineplot(data=D, x = 'cond_num', y = 'Y_norm', hue = 'roi_name',style='roi_name',
#             palette=pal,
#             dashes=[d1,d1,d1,d1,d2,d2,d2,d2],
#             err_style=None)
# # Find mapping between cond_name and cond_num
# ax.set_xticks(np.arange(12)+1)
# l=ax.set_xticklabels(cond_map.cond_name.values, rotation=45)

DD = plotting.roi_difference(D, cond_map, depvar = "Y_norm", var = ["cond_name","roi_name"])


ValueError: Length of DataFrame vectors must match length of `data` when both are used, but `data` has length 3072 and the vector passed to `x` has length 12.

<Figure size 1200x800 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:

A = pd.pivot_table(data=D,index='roi_name',columns='cond_name',values='Y_norm',aggfunc=np.mean)
C=A.values
C=C/np.sqrt((C**2).sum(axis=1,keepdims=True))
B = C@C.T
K=3
W,V = plotting.calc_mds(A.values,K=K,center=True)
# phase, load, and recall
vs = np.array([[-1, 1,-1, 1,-1,1,-1,1,-1,1,-1,1],
                [-1,-1,-1,-1, 0,0, 0,0, 1,1, 1,1],
                [1,1, -1, -1, 1, 1, -1, -1,1,1, -1, -1]])
vs = vs/np.sqrt((vs**2).sum(axis=1,keepdims=True))
proj_vs = V @ vs.T

pal = [red,red,gray,gray,lb,lb,db,db]

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plotting.plot_mds(W[:,0],W[:,1],A.index,
                      colors=pal,
                      vectors=proj_vs[:2,:],
                      v_labels = ['retrieval','load+','backwards'])
plt.subplot(1,2,2)
plotting.plot_mds(W[:,2],W[:,1],A.index,
                      colors=pal,
                      vectors=proj_vs[[2,1],:],
                      v_labels = ['retrieval','load+','backwards'])


In [ ]:
plotting.plot_mds3(W[:,0],W[:,1], W[:,1], 
                   A.index, 
                   colors=pal,
                   text_size = 'small', 
                   text_weight = 'regular',
                   vectors = proj_vs,
                   v_labels = ['retrieval','load+','backwards'])

## Parcellation divided into anterior and posterior

In [ ]:
df_path = os.path.join(wkdir, "ROI_NettekovenSym68c32AP_conn_reg.tsv")
D = pd.read_csv(df_path, sep="\t")
D, cond_map = prep_roi_comparison(D)

In [ ]:
D = norm_within_category(D, category=['roi_name','sn'], value='Y', norm='mean')
D = D.loc[D.roi != 0]
D = D.loc[D.cond_name != "rest"]
D["AP"] = D["roi_name"].str[4]
plt.figure(figsize=(12,8))
# Define styles and colors
d1 = (1,0)
d2 = (3,3)
redA =(0.8,0.2,0.2)
redP =(0.6,0.3,0.)
grayA = (0.5,0.5,0.5)
grayP = (0.1,0.1,0.1)
lbA = (0.2,0.5,1.0)
lbP = (0.2, 0.7, 1.0)
dbA = (0.0,0.1,0.6)
dbP = (0.0, 0.4, 0.6)

DD = plot_roi_differences(D, cond_map, depvar = "Y_norm", var = ["cond_name", "AP"])


In [ ]:
A = pd.pivot_table(data=D,index='roi_name',columns='cond_name',values='Y_norm',aggfunc=np.mean)
C=A.values
C=C/np.sqrt((C**2).sum(axis=1,keepdims=True))
B = C@C.T

K=3
W,V = plotting.calc_mds(A.values,K=K,center=True)
# phase, load, and recall
vs = np.array([[-1, 1,-1, 1,-1,1,-1,1,-1,1,-1,1],
                [-1,-1,-1,-1, 0,0, 0,0, 1,1, 1,1],
                [1,1, -1, -1, 1, 1, -1, -1,1,1, -1, -1]])
vs = vs/np.sqrt((vs**2).sum(axis=1,keepdims=True))
proj_vs = V @ vs.T

pal = [redA,redA,redP,redP,grayA,grayA,grayP,grayP,lbA,lbA,lbP,lbP,dbA,dbA,dbP,dbP]

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plotting.plot_mds(W[:,0],W[:,1],A.index,
                      colors=pal,
                      vectors=proj_vs[:2,:],
                      v_labels = ['retrieval','load+','backwards'])
plt.subplot(1,2,2)
plotting.plot_mds(W[:,2],W[:,1],A.index,
                      colors=pal,
                      vectors=proj_vs[[2,1],:],
                      v_labels = ['retrieval','load+','backwards'])

In [ ]:
plotting.plot_mds3(W[:,0],W[:,1], W[:,1], 
                   A.index, 
                   colors=pal,
                   text_size = 'small', 
                   text_weight = 'regular',
                   vectors = proj_vs,
                   v_labels = ['retrieval','load+','backwards'])